In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

In [5]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informação geral e detalhada no google sobre um tópico.",
        tools=[google_search],
        instruction="""
       Função: Agente especializado em buscar e coletar dados e informações
       relevantes em (Google Search).
       Fornecer pesquisa detalhadas e relevantes para o planejamento e organização de conteúdo.
       Oferecer apenas as pesquisas mais relevantes.
       **Instruções de Busca Específicas:**
       1. **Foco Principal:** Busque por **informações aprofundadas, dados, fatos, tendências, contextos e detalhes** sobre o tópico fornecido. O objetivo é obter um entendimento completo e factual do assunto.
       2. **Conteúdo a Excluir Ativamente:**
          - Cursos online, treinamentos, webinars, plataformas de ensino à distância, materiais de curso.
       3. **Recomendações (Livros, etc.):** Se encontrar recomendações relevantes (como livros) que **complementam ou ilustram a informação geral** sobre o tópico, pode incluí-las. No entanto, o foco **não** é criar uma lista de recomendações, mas sim usar essas recomendações como exemplos ou fontes dentro de um contexto informativo mais amplo.

       Identificar e extrair informações chave, dados, tendências e exemplos **que expliquem e detalhem o tópico**.
       Summarizar os achados de forma clara e concisa, priorizando a apresentação das **informações informativas e factuais** encontradas. Mencione recomendações apenas se forem altamente relevantes para o conteúdo informativo.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    resposta_do_agente_buscador = call_agent(buscador, entrada_do_agente_buscador)
    return resposta_do_agente_buscador

In [10]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Função: Agente especializado em estruturar e organizar informações para
        criar planos de conteúdo detalhados e acionáveis para a redação final.
        Baseado no Tópico e nos dados de Lançamentos buscados, crie um plano
        de conteúdo organizado em SEÇÕES ENUMERADAS em Markdown.

        Exemplo de formato desejado:
        1. Título da Primeira Seção (ex: tema 1)
        2. Título da Segunda Seção (ex: tema 2)
        3. Título da Terceira Seção (ex: tema 3)
        4. Titulo da Quarta Seção (ex: tema 4)

        Certifique-se de que cada ponto numerado é um título de seção claro e conciso.
        Filtrar os dados recebidos pelo agente_buscador selecionando apenas 4 mais relevantes e importantes
        abordar.
        NÃO GERAR MAIS DE 4 SEÇÕES.
        Selecionar sitações, pesquisas ou fatos interessantes para abordar nos paragrafos.
        O plano deve ser uma lista numerada que delineia a estrutura completa do post.
        """,
        description="Planejador de conteúdo",
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [11]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático.
        Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
        escreva o texto abordando o tema.
        Importante ressaltar para deixar pelo menos 1 citação relevante.
        O post deve ser engajador, informativo, que prenda a tenção e foco do leitor.
        O texto deve seguir um padrão de no máximo 300 palavras.
            """,
        description="Agente redator de posts engajador"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [12]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um revisor Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático. Seja conciso em suas redações.
        Revise o rascunho de post abaixo sobre o tópico indicado, verificando
        sua escrita.
        Não desejo que seja exigente e nem faça muitas correções
        Apenas exija mudança se for realmente importante e necessário para
        melhoria do material original.
        Se o rascunho estiver bom, responda apenas 'O rascunho está pronto!'.
        O texto deve seguir um padrão de no máximo 300 palavras.
        proibido de fazer correção após 2 tentativas.
        """,
        description="Agente revisor de post."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [13]:
from datetime import date
import re
from IPython.display import display, Markdown
from google.adk.agents import Agent  # Import Agent
from google.adk.runners import Runner # Import Runner
from google.adk.sessions import InMemorySessionService # Import InMemorySessionService
from google.adk.tools import google_search # Import google_search
from google.genai import types  # Para criar conteúdos (Content e Part) # Import types
import textwrap

# --- Funções Auxiliares (mantenha ou adapte conforme necessário) ---
# ... (mantenha as definições de extract_content_from_part, call_agent, to_markdown, extrair_secoes_do_plano)

def extract_content_from_part(part):
    """Extracts text content from a GenAI Part."""
    if part.text:
        return part.text
    return ""

def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    try:
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            if event.is_final_response():
              for part in event.content.parts:
                final_response += extract_content_from_part(part)
                final_response += "\n"
    except Exception as e:
        print(f"Error during agent run: {e}")
    return final_response

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


def extrair_secoes_do_plano(texto_plano: str) -> list[str]:
    secoes = []
    padroes_secao = [
        r'^\s*##\s*(.+)',
        r'^\s*\d+\.\s*(.+)',
        r'^\s*-\s*(.+)'
    ]
    linhas = texto_plano.splitlines()
    for linha in linhas:
        for padrao in padroes_secao:
            match = re.match(padrao, linha.strip())
            if match:
                secao_titulo = match.group(1).strip()
                if secao_titulo and len(secao_titulo) > 3:
                    secoes.append(secao_titulo)
                break
    if not secoes and texto_plano.strip():
         return ["Conteúdo Principal"]
    elif not secoes and not texto_plano.strip():
        return []
    return secoes


# --- Agente 1: Buscador de Notícias ---
# Defina o objeto do agente aqui ou antes da lógica principal
buscador_agent_obj = Agent(
    name="agente_buscador",
    model="gemini-2.0-flash",
    description="Agente que busca informação geral e detalhada no google sobre um tópico.",
    tools=[google_search],
    instruction="""
   Função: Agente especializado em buscar e coletar dados e informações
   relevantes em (Google Search).
   Fornecer pesquisa detalhadas e relevantes para o planejamento e organização de conteúdo.
   Oferecer apenas as pesquisas mais relevantes.
   **Instruções de Busca Específicas:**
   1. **Foco Principal:** Busque por **informações aprofundadas, dados, fatos, tendências, contextos e detalhes** sobre o tópico fornecido, **considerando o foco e propósito especificados pelo usuário**. O objetivo é obter um entendimento completo e factual do assunto, **alinhado com a forma de abordagem desejada**.
   2. **Conteúdo a Excluir Ativamente:**
      - Cursos online, treinamentos, webinars, plataformas de ensino à distância, materiais de curso.
   3. **Recomendações (Livros, etc.):** Se encontrar recomendações relevantes (como livros) que **complementam ou ilustram a informação geral** sobre o tópico, pode incluí-las. No entanto, o foco **não** é criar uma lista de recomendações, mas sim usar essas recomendações como exemplos ou fontes dentro de um contexto informativo mais amplo.

   Identificar e extrair informações chave, dados, tendências e exemplos **que expliquem e detalhem o tópico**, **seguindo a forma de abordagem e o propósito definidos pelo usuário**.
   Summarizar os achados de forma clara e concisa, priorizando a apresentação das **informações informativas e factuais** encontradas. Mencione recomendações apenas se forem altamente relevantes para o conteúdo informativo.
    """
)

# --- Agente 2: Planejador de posts ---
# Defina o objeto do agente aqui ou antes da lógica principal
planejador_agent_obj = Agent(
    name="agente_planejador",
    model="gemini-2.0-flash",
    instruction="""
    Função: Agente especializado em estruturar e organizar informações para
    criar planos de conteúdo detalhados e acionáveis para a redação final.
    Baseado no Tópico, Forma de Abordagem, Propósito e nos dados de Lançamentos buscados, crie um plano
    de conteúdo organizado em SEÇÕES ENUMERADAS em Markdown.

    Exemplo de formato desejado:
    1. Título da Primeira Seção (ex: tema 1)
    2. Título da Segunda Seção (ex: tema 2)
    3. Título da Terceira Seção (ex: tema 3)
    4. Titulo da Quarta Seção (ex: tema 4)

    Certifique-se de que cada ponto numerado é um título de seção claro e conciso.
    Filtrar os dados recebidos pelo agente_buscador selecionando apenas 4 mais relevantes e importantes
    abordar, **considerando a Forma de Abordagem e o Propósito especificados**.
    NÃO GERAR MAIS DE 4 SEÇÕES.
    Selecionar sitações, pesquisas ou fatos interessantes para abordar nos paragrafos, **alinhados com a Forma de Abordagem e Propósito**.
    O plano deve ser uma lista numerada que delineia a estrutura completa do post.
    """,
    description="Planejador de conteúdo",
)

# --- Agente 3: Redator do Post ---
# Defina o objeto do agente aqui ou antes da lógica principal
redator_agent_obj = Agent(
    name="agente_redator",
    model="gemini-2.0-flash",
    instruction="""
        Você é um redator Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático.
        Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
        escreva o texto abordando o tema, **mantendo a Forma de Abordagem e o Propósito originais**.
        Importante ressaltar para deixar pelo menos 1 citação relevante.
        O post deve ser engajador, informativo, que prenda a tenção e foco do leitor.
        O texto deve seguir um padrão de no máximo 300 palavras.

        """,
    description="Agente redator de posts engajador"
)

# --- Agente 4: Revisor de Qualidade ---
# Defina o objeto do agente aqui ou antes da lógica principal
revisor_agent_obj = Agent(
    name="agente_revisor",
    model="gemini-2.0-flash",
    instruction="""
        Você é um revisor Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático. Seja conciso em suas redações.
        Revise o rascunho de post abaixo sobre o tópico indicado, verificando
        sua escrita, **assegurando que a Forma de Abordagem e o Propósito originais sejam mantidos**.
        Não desejo que seja exigente e nem faça muitas correções
        Apenas exija mudança se for realmente importante e necessário para
        melhoria do material original.
        Se o rascunho estiver bom, responda apenas 'O rascunho está pronto!'.
        O texto deve seguir um padrão de no máximo 300 palavras.
        proibido de fazer correção após 2 tentativas.
        """,
    description="Agente revisor de post."
)


# --- Lógica Principal de Orquestração do Chatbot (Modificada) ---

## 🚀 Iniciando o Sistema de Planejamento e Geração de Conteúdo 🚀

# Obtém a data atual para uso posterior
data_de_hoje = date.today().strftime("%d/%m/%Y")

### ❓ Obtendo o Tópico e Preferências do Usuário

print("## 📚 Inicie o Planejamento do seu Conteúdo! 📚")

print("\nPor favor, digite o TÓPICO principal sobre o qual você quer estudar/criar conteúdo:")
topico = input("Tópico: ")

if not topico:
  print("❌ Nenhum tópico inserido. Por favor, insira um tópico válido para continuar.")
else:
  print(f"\n✅ Tópico recebido: {topico}.")

  print("\nCom qual FORMA DE ABORDAGEM você deseja que o tema seja tratado? (Ex: Técnica, Didática, Histórica, Comparativa, etc.)")
  forma_abordagem = input("Forma de Abordagem: ")

  print("\nQual o PROPÓSITO principal deste conteúdo? (Ex: Explicar um conceito, Comparar tecnologias, Apresentar tendências, Guia prático, etc.)")
  proposito_conteudo = input("Propósito do Conteúdo: ")

  print(f"\n✅ Preferências capturadas:")
  print(f"- Tópico: {topico}")
  print(f"- Forma de Abordagem: {forma_abordagem if forma_abordagem else 'Não especificada'}")
  print(f"- Propósito: {proposito_conteudo if proposito_conteudo else 'Não especificado'}")

  print("\nIniciando o processo de busca e planejamento...")

  ## 🔍 Etapa 1: Buscando Informações Relevantes (Agente Buscador)

  print("\n--- Executando Agente Buscador ---")
  # Chama o agente buscador para coletar dados sobre o tópico
  # Inclua as preferências do usuário na entrada do agente buscador
  entrada_do_agente_buscador = f"""
  Tópico: {topico}
  Data de hoje: {data_de_hoje}
  Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
  Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
  """
  lançamentos_buscados = call_agent(buscador_agent_obj, entrada_do_agente_buscador)

  print("\n### Resultado da Busca:")
  # Exibe os resultados da busca formatados em Markdown
  display(to_markdown(lançamentos_buscados))
  print("-------------------------------")

  ## 📝 Etapa 2: Planejando a Estrutura do Conteúdo (Agente Planejador)

  print("\n--- Executando Agente Planejador ---")
  # Chama o agente planejador para criar um plano de conteúdo baseado na busca
  # Inclua as preferências do usuário na entrada do agente planejador
  entrada_do_agente_planejador = f"""
  Tópico: {topico}
  Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
  Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
  Lançamentos buscados: {lançamentos_buscados}
  """
  plano_de_post = call_agent(planejador_agent_obj, entrada_do_agente_planejador)

  print("\n### Plano de Conteúdo Gerado:")
  # Exibe o plano de conteúdo formatado em Markdown
  display(to_markdown(plano_de_post))
  print("-------------------------------")

  ## ✍️ Etapa 3: Gerando e Revisando Conteúdo por Seção (Agente Redator & Revisor)

  print("\n--- Iniciando Geração e Revisão do Rascunho (por Seção) ---")

  # Extrai as seções do plano de conteúdo para processamento individual
  secoes_do_plano = extrair_secoes_do_plano(plano_de_post)

  # Crie uma lista para armazenar os rascunhos de cada seção
  rascunhos_individuais_secoes = []

  if not secoes_do_plano:
      print("⚠️ Aviso: Não foi possível extrair seções do plano. Não será possível gerar o rascunho por seção.")
      rascunho_do_post = "" # Define rascunho_do_post como vazio se não houver seções
  else:
      rascunho_do_post_partes = [] # Lista para armazenar as partes do rascunho por seção
      MAX_TENTATIVAS_REVISAO = 3 # Limite para o ciclo de revisão por seção

      # Loop sobre cada seção do plano
      for i, secao in enumerate(secoes_do_plano):
          print(f"\n--- Processando Seção {i+1}/{len(secoes_do_plano)}: '{secao}' ---")

          ### ✏️ Gerando Rascunho Inicial da Seção

          # Inclua as preferências do usuário na entrada para o redator
          entrada_para_redator_secao = f"""
          Tópico Geral: {topico}
          Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
          Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}

          Plano de Conteúdo Completo:
          {plano_de_post}

          ---
          POR FAVOR, ESCREVA APENAS O CONTEÚDO DA SEÇÃO ESPECÍFICA ABAIXO.
          Não inclua o título da seção no início da sua resposta.
          Seção a escrever: {secao}

          Instruções Adicionais para esta Seção:
          (Se houver instruções específicas para esta seção do plano, adicione aqui)
          """
          rascunho_da_secao = call_agent(redator_agent_obj, entrada_para_redator_secao)
          print("\n#### Rascunho Inicial:")
          display(to_markdown(rascunho_da_secao))
          print("-------------------------------")

          ### 👀 Ciclo de Revisão da Seção

          revisao_finalizada = False
          tentativas_revisao = 0

          # Loop de revisão até aprovação ou limite de tentativas
          while not revisao_finalizada and tentativas_revisao < MAX_TENTATIVAS_REVISAO:
              tentativas_revisao += 1
              print(f"\n--- Revisando Seção (Tentativa {tentativas_revisao}/{MAX_TENTATIVAS_REVISAO}) ---")

              # Inclua as preferências do usuário na entrada para o revisor
              entrada_para_revisor_secao = f"""
              Tópico Geral: {topico}
              Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
              Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
              Plano de Conteúdo Relevante para esta Seção: {plano_de_post}

              ---
              Rascunho da Seção "{secao}" para Revisão:
              {rascunho_da_secao}

              Por favor, revise o rascunho da seção acima. Se estiver bom, responda APENAS a frase exata 'O rascunho está ótimo e pronto!'. Caso contrário, aponte os problemas e sugira melhorias específicas para que o redator possa corrigir. Certifique-se de que a revisão leva em conta a Forma de Abordagem e o Propósito originais.
              """

              feedback_revisor = call_agent(revisor_agent_obj, entrada_para_revisor_secao)
              print("\n#### Feedback do Revisor:")
              display(to_markdown(feedback_revisor))
              print("-------------------------------")

              if feedback_revisor.strip().lower() == 'o rascunho está ótimo e pronto!':
                  print(f"👍 Seção '{secao}' aprovada pelo Revisor!")
                  revisao_finalizada = True # Sai do loop de revisão para esta seção
              else:
                  print(f"🔄 Seção '{secao}' precisa de correções. Enviando feedback para o Redator.")
                  # Prepara a entrada para o Redator corrigir com base no feedback
                  # Inclua as preferências do usuário na entrada para o redator corrigir
                  entrada_para_redator_correcao = f"""
                  Tópico Geral: {topico}
                  Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
                  Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
                  Plano de Conteúdo Completo:
                  {plano_de_post}
                  ---
                  POR FAVOR, REESCREVA/CORRIJA APENAS O CONTEÚDO DA SEÇÃO ESPECÍFICA ABAIXO, BASEADO NO FEEDBACK.
                  Não inclua o título da seção no início da sua resposta.
                  Seção a corrigir: {secao}
                  Rascunho Anterior Desta Seção:
                  {rascunho_da_secao}
                  ---
                  Feedback do Revisor para Correção:
                  {feedback_revisor}
                  Por favor, incorpore as sugestões de correção do revisor no novo rascunho desta seção, garantindo que a Forma de Abordagem e o Propósito sejam mantidos.
                  """
                  # Chama o Redator novamente com a solicitação de correção
                  rascunho_da_secao = call_agent(redator_agent_obj, entrada_para_redator_correcao)
                  print("\n#### Rascunho Corrigido:")
                  display(to_markdown(rascunho_da_secao))
                  print("-------------------------------")

          # Fim do ciclo de revisão para a seção atual

          if not revisao_finalizada:
              print(f"⚠️ Aviso: A Seção '{secao}' não foi aprovada após {MAX_TENTATIVAS_REVISAO} tentativas. Usando a última versão gerada.")

          # Adiciona a versão final da seção (aprovada ou a última tentativa) à lista de rascunhos individuais
          # Adiciona também o título da seção para melhor visualização no final
          rascunhos_individuais_secoes.append(f"## {secao}\n\n{rascunho_da_secao}")

          # Adiciona também às partes do rascunho completo (mantendo a funcionalidade original)
          rascunho_do_post_partes.append(f"## {secao}\n\n{rascunho_da_secao}")


      # Junta todas as partes das seções revisadas para formar o rascunho completo
      rascunho_do_post = "\n\n".join(rascunho_do_post_partes)

      print("\n--- Geração e Revisão por Seção Concluídas ---")
      print("\n## 🎉 Rascunho Completo Gerado (Após Revisão por Seção) 🎉")
      # Exibe o rascunho completo final formatado
      display(to_markdown(rascunho_do_post))
      print("-------------------------------")


  ## ✅ Processo de Planejamento e Geração Finalizado ✅

  print("\nO sistema concluiu o processo de planejamento e geração de conteúdo.")

## 📚 Inicie o Planejamento do seu Conteúdo! 📚

Por favor, digite o TÓPICO principal sobre o qual você quer estudar/criar conteúdo:
Tópico: Biologia molecular

✅ Tópico recebido: Biologia molecular.

Com qual FORMA DE ABORDAGEM você deseja que o tema seja tratado? (Ex: Técnica, Didática, Histórica, Comparativa, etc.)
Forma de Abordagem: Didatica

Qual o PROPÓSITO principal deste conteúdo? (Ex: Explicar um conceito, Comparar tecnologias, Apresentar tendências, Guia prático, etc.)
Propósito do Conteúdo: resumo para estudo

✅ Preferências capturadas:
- Tópico: Biologia molecular
- Forma de Abordagem: Didatica
- Propósito: resumo para estudo

Iniciando o processo de busca e planejamento...

--- Executando Agente Buscador ---

### Resultado da Busca:


> Ok, entendi. Vou buscar informações detalhadas sobre biologia molecular, com foco em dados, fatos, tendências e exemplos relevantes para um resumo de estudo com abordagem didática.
> 
> 
> Com certeza! Aqui está um resumo didático sobre biologia molecular para seus estudos:
> 
> **O que é Biologia Molecular?**
> 
> *   É o ramo da biologia que estuda a vida em nível molecular.
> *   Examina a estrutura, a função e as interações entre as moléculas biológicas (DNA, RNA e proteínas) dentro das células.
> *   Busca entender como esses componentes moleculares influenciam os processos biológicos e as características dos organismos vivos.
> 
> **Conceitos Fundamentais:**
> 
> *   **DNA (Ácido Desoxirribonucleico):**
>     *   É a molécula que contém as informações genéticas.
>     *   Possui uma estrutura de dupla hélice composta por nucleotídeos (adenina, timina, citosina e guanina).
> *   **RNA (Ácido Ribonucleico):**
>     *   Atua em diversas funções celulares, incluindo a síntese de proteínas.
>     *   Existem diferentes tipos de RNA (mRNA, tRNA, rRNA) com papéis específicos.
> *   **Proteínas:**
>     *   São moléculas complexas formadas por aminoácidos.
>     *   Desempenham diversas funções nas células, como catálise de reações, transporte de moléculas e estruturação celular.
> *   **Dogma Central da Biologia Molecular:**
>     *   Descreve o fluxo de informação genética: DNA → RNA → Proteína.
>     *   O DNA é transcrito em RNA, que é traduzido em proteínas.
> 
> **Processos Essenciais:**
> 
> *   **Replicação:**
>     *   Processo de cópia do DNA para garantir a transmissão da informação genética durante a divisão celular.
>     *   Enzimas como a DNA polimerase desempenham um papel fundamental.
> *   **Transcrição:**
>     *   Processo de produção de RNA a partir de um molde de DNA.
>     *   A enzima RNA polimerase é responsável por esse processo.
> *   **Tradução:**
>     *   Processo de síntese de proteínas a partir da informação contida no RNA mensageiro (mRNA).
>     *   Ocorre nos ribossomos, com a participação do tRNA e rRNA.
> *   **Regulação da Expressão Gênica:**
>     *   Mecanismos que controlam quais genes são expressos em determinados momentos e em quais células.
>     *   Envolve interações entre proteínas e sequências específicas de DNA.
> 
> **Técnicas Importantes:**
> 
> *   **PCR (Reação em Cadeia da Polimerase):**
>     *   Técnica para amplificar (criar muitas cópias) de um segmento específico de DNA.
>     *   Amplamente utilizada em diagnósticos, pesquisa e forense.
> *   **Eletroforese:**
>     *   Método para separar moléculas (DNA, RNA ou proteínas) com base em seu tamanho e carga elétrica.
> *   **Clonagem:**
>     *   Processo de produção de cópias idênticas de um fragmento de DNA.
> *   **Sequenciamento de DNA:**
>     *   Determinação da ordem dos nucleotídeos em uma molécula de DNA.
>     *   Fundamental para identificar genes, mutações e variações genéticas.
> *   **CRISPR-Cas9:**
>     *   Tecnologia de edição genética que permite modificar genes de forma precisa.
> 
> **História da Biologia Molecular:**
> 
> *   **1869:** Friedrich Miescher isola o ácido nucleico.
> *   **1866:** Gregor Mendel inventou e desenvolveu a lei da herança (conhecida como lei das ervilhas).
> *   **1953:** James Watson e Francis Crick propõem o modelo da dupla hélice do DNA.
> *   **Década de 1980:** Desenvolvimento da técnica de PCR.
> *   **2003:** Conclusão do sequenciamento do genoma humano.
> 
> **Aplicações da Biologia Molecular:**
> 
> *   **Medicina:**
>     *   Diagnóstico de doenças genéticas e infecciosas.
>     *   Desenvolvimento de terapias gênicas e medicamentos personalizados.
>     *   Identificação de agentes infecciosos
> *   **Agricultura:**
>     *   Melhoramento genético de plantas para aumentar a produtividade e resistência a pragas.
>     *   Produção de culturas transgênicas.
> *   **Biotecnologia:**
>     *   Produção de proteínas recombinantes para uso terapêutico e industrial.
>     *   Desenvolvimento de novas tecnologias para análise e manipulação do material genético.
> *   **Ciência Forense:**
>     *   Análise de DNA para identificação de indivíduos em investigações criminais.
>     *   Resolução de casos legais.
> 
> **Reagentes:**
> 
> *   Enzimas de restrição.
> *   DNA e RNA polimerases.
> *   Nucleotídeos.
> *   Ligases.
> 
> Espero que este resumo seja útil para seus estudos!
> 


-------------------------------

--- Executando Agente Planejador ---

### Plano de Conteúdo Gerado:


> 1.  Conceitos Fundamentais da Biologia Molecular: DNA, RNA e Proteínas
> 2.  Processos Essenciais: Replicação, Transcrição e Tradução
> 3.  Técnicas Fundamentais da Biologia Molecular: PCR, Eletroforese e CRISPR-Cas9
> 4.  Aplicações Práticas da Biologia Molecular: Medicina, Agricultura e Biotecnologia


-------------------------------

--- Iniciando Geração e Revisão do Rascunho (por Seção) ---

--- Processando Seção 1/4: 'Conceitos Fundamentais da Biologia Molecular: DNA, RNA e Proteínas' ---

#### Rascunho Inicial:


...

-------------------------------


> ## Técnicas Fundamentais da Biologia Molecular: PCR, Eletroforese e CRISPR-Cas9
> 
> Na biologia molecular, algumas técnicas são tão importantes que se tornaram verdadeiros pilares. A **reação em cadeia da polimerase (PCR)**, por exemplo, é como uma fotocopiadora de DNA, permitindo amplificar milhões de cópias de um segmento específico. Já a **eletroforese** funciona como uma peneira molecular, separando moléculas de DNA, RNA ou proteínas com base no tamanho e carga elétrica.
> 
> E não podemos esquecer da edição gênica com o sistema **CRISPR-Cas9**, uma ferramenta revolucionária que funciona como um "Ctrl+C Ctrl+V" do DNA, permitindo editar genes com uma precisão nunca antes vista.
> 
> _"A biologia molecular não é apenas uma disciplina, mas uma abordagem"_ - **Sydney Brenner**.


-------------------------------


> ## Aplicações Práticas da Biologia Molecular: Medicina, Agricultura e Biotecnologia
> 
> Na medicina, a biologia molecular revolucionou o diagnóstico e tratamento de doenças. Testes genéticos identificam predisposições, personalizando terapias. A terapia gênica oferece esperança para doenças genéticas, corrigindo ou substituindo genes defeituosos.
> 
> Na agricultura, a biologia molecular cria plantas mais resistentes e nutritivas. A modificação genética de culturas, embora debatida, visa aumentar a produção de alimentos e reduzir o uso de pesticidas.
> 
> Na biotecnologia, a biologia molecular impulsiona a produção de biofármacos, enzimas e produtos industriais. A engenharia de microrganismos para biocombustíveis e bioplásticos exemplifica aplicações inovadoras para um futuro sustentável.
> 
> "A biologia molecular é a chave para compreendermos a vida em sua essência e para desenvolvermos soluções inovadoras para os desafios que enfrentamos." - Sydney Brenner, ganhador do Prêmio Nobel de Fisiologia ou Medicina.
> 


-------------------------------

--- Fim da Visualização dos Rascunhos Individuais por Seção ---


In [17]:
!pwd

/content


In [18]:
%cd /content

!git clone https://github.com/Thomas-Yano/Alura_projeto
!git config --global user.name "Thomas-Yano"
!git config --global user.email "yanoyuiti2019@gmail.com"
!git add .
!git commit -m "Primeiro commit: Adicionando o notebook do projeto"
!git remote add origin <URL_DO_SEU_REPOSITORIO_GITHUB>
!git push -u origin main


/content
Cloning into 'Alura_projeto'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 34.19 KiB | 3.42 MiB/s, done.
Resolving deltas: 100% (3/3), done.
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `git remote add origin <URL_DO_SEU_REPOSITORIO_GITHUB>'
fatal: not a git repository (or any of the parent directories): .git
